In [5]:
from PyPDF2 import PdfReader
import re
import json

In [7]:
reader = PdfReader('info.pdf')

In [29]:
def parse_text(text):
    # we get the first part of the question
    # until the word algorithm
    question_match = re.search(r'(.*?)(?=Algor)', text, re.DOTALL)
    question_before = question_match.group(1).strip() if question_match else ''

    # we get the code, from algo-endalgo
    code_match = re.search(r'(Algor.*?EndAlgor)', text, re.DOTALL)
    code = code_match.group(1).strip() if code_match else ''

    # leftover question, after code
    question_after_match = re.search(r'EndAlgor(.*?)(?=[A-Z]\.|$)', text, re.DOTALL)
    question_after = question_after_match.group(1).strip() if question_after_match else ''

    question = " ".join([question_before, question_after])

     # answers
    answers_match = re.findall(r'([A-Z])\.(.*?)(?=[A-Z]\.|$)', text, re.DOTALL)
    answers = [{key: value.strip()} for key, value in answers_match] if answers_match else []

    # Create dictionary
    parsed_dict = {
        "question": question,
        "answers": answers,
        "code": code
    }

    return parsed_dict


In [17]:
text = ""

for i in range(len(reader.pages)):
    page = reader.pages[i]

    page_text = page.extract_text()
    text += page_text

In [21]:
# regex to split questions
# all start with a number then a dot
questions = re.split(r'\n(?=\d+\.)', text)

In [30]:
jsonDump = []

for question in questions:
    if question.strip():
        jsonDump.append(parse_text(question))

In [31]:
jsonDump = json.dumps(jsonDump, indent=4, ensure_ascii=False)
print(jsonDump)

[
    {
        "question": " ",
        "answers": [],
        "code": ""
    },
    {
        "question": " ithm  Which of the following function calls will return 4?",
        "answers": [
            {
                "A": "F(21369)"
            },
            {
                "B": "F(6933)"
            },
            {
                "C": "F(4)"
            },
            {
                "D": "F(16639)"
            }
        ],
        "code": "Algorithm  F(x):  \n    If x = 0 then \n        Return 0 \n    Else \n        If x MOD 3 = 0 then \n            Return F(x DIV 10) + 1  \n        Else \n            Return F(x DIV 10) \n        EndIf  \n    EndIf  \nEndAlgor"
    },
    {
        "question": " itm  The algorithm  ceFace(a, b)  returns  True  if and only if :",
        "answers": [
            {
                "A": "a and b are equal"
            },
            {
                "B": "a and b are palindromes"
            },
            {
                "C": "a is the r